In [ ]:
!pip install evaluate sacrebleu bert_score

In [ ]:
!pip install -U bitsandbytes

In [1]:
import pandas as pd
import os, json, math, random, re, html
from dataclasses import dataclass
from typing import Dict, List, Optional, Union

import datasets
from datasets import load_dataset, DatasetDict
import evaluate
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    set_seed,
)

# PEFT/LoRA opcionais
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"

2025-11-14 22:34:15.508038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763159655.531032     577 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763159655.538280     577 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

(True, 2, 'Tesla T4')

In [2]:
# ===== CONFIGURAÇÕES EDITÁVEIS =====
PROJECT_NAME    = "ptt5v2-pl-text2action"
DATA_PATH       = "/kaggle/input/pl-to-action-dataset/pl_action_recommendations_all.jsonl"
OUTPUT_DIR      = "./outputs_ptt5v2"              # onde salvar checkpoints
EVAL_SPLIT      = 0.1                              # fração para validação
MAX_INPUT_LEN   = 256
MAX_TARGET_LEN  = 32

# Escolha do modelo base (PTT5-v2). Alguns checkpoints comuns:
# - "unicamp-dl/ptt5-base-portuguese-vocab"  (mais conhecido)
# - "pierreguillou/ptt5-base-portuguese-vocab" (espelho)
# Se você já tem um "PTT5-v2" específico, coloque o ID abaixo.
MODEL_ID        = "unicamp-dl/ptt5-base-portuguese-vocab"

# Hiperparâmetros sugeridos (ajuste conforme sua GPU)
SEED            = 42
BATCH_SIZE      = 16
GRAD_ACC_STEPS  = 2
LR              = 3e-4
EPOCHS          = 30
FP16            = torch.cuda.is_available()
BF16            = False   # Ative se a sua GPU suportar (A100/A800/H100/RTX 5xxx)
WARMUP_RATIO    = 0.03
WEIGHT_DECAY    = 0.01

# QLoRA (8-bit/4-bit) opções
USE_4BIT        = True    # True = quantização 4-bit (menos VRAM); False = 8-bit
LORA_R          = 16
LORA_ALPHA      = 32
LORA_DROPOUT    = 0.05

# Template simples de instrução (pode adaptar)
INSTR_PROMPT = (
  "Converta a ementa de projeto de lei em uma recomendação de ação imperativa, curta e fiel ao texto; "
  "{texto}\nSaída:"
)

In [ ]:
INSTR_PROMPT = (
    "Dada a ementa de um projeto de lei em linguagem jurídica, gere uma única recomendação de ação operacional em português no formato [Verbo no infinitivo] + [objeto] + [complementos essenciais], por exemplo: “Implantar estufas com hortas produzidas com garrafas PET nas escolas municipais.” Remova toda a “casca jurídica” que não muda a ação (“Dispõe sobre…”, “Institui…”, “Cria…”, “Autoriza o Poder Executivo a…”, “e dá outras providências”, referências a leis, artigos e fórmulas padrão), preservando apenas o conteúdo material da política: o que passa a existir, ser feito, fornecido ou garantido. Identifique o núcleo da ementa (substantivos de ação como criação, implantação, emissão, fornecimento, atendimento etc.) e transforme-o em verbo no infinitivo impessoal (criar, implantar, emitir, fornecer, atender etc.), seguido do objeto principal e dos complementos realmente necessários (público-alvo e/ou local, quando essenciais para entender a execução). Quando a ementa criar um equipamento, serviço, órgão ou programa (inclusive digitais), a ação deve ser “criar” ou “implantar” esse instrumento; quando houver estrutura do tipo “Programa/Projeto X para [substantivo de ação]…”, priorize o serviço final (ex.: “emissão de registro de nascimento” → “Emitir registros de nascimento dentro das maternidades públicas”) e não o programa em si. Neutralize nomes fantasia de programas (“Segurinho”, “Saúde ao Alcance” etc.), descrevendo-os de forma genérica pelo tipo de programa/serviço, a menos de datas comemorativas, prêmios, selos ou eventos culturais, em que o nome é o próprio objeto e deve ser mantido. Elimine justificativas, fundamentos legais e detalhes que não alteram a execução, produzindo sempre uma frase imperativa, curta e operacional, algo que caiba em um backlog de políticas públicas."
    "\n\nEmenta: {texto}\n\nAção: "
)

print(INSTR_PROMPT)

In [3]:
def normalize_text(s: str) -> str:
    # Remove entidades HTML, que aparecem com frequência no seu exemplo (&#8211;, &#8220; etc.)
    s = html.unescape(s)
    # Quebras de linha e espaços
    s = s.replace("\r", " ").replace("\n", " ").strip()
    # Espacos múltiplos
    s = re.sub(r"\s+", " ", s)
    return s

data = pd.read_csv("/kaggle/input/ementa2action-dataset/ementa2action_gpt.csv")
data = data.iloc[:, 1:]
data = data.rename(columns={"ementa": "input", "acao": "output"})
raw = list(data.T.to_dict().values())

for r in raw:
    r["input"]  = normalize_text(r["input"].lower())
    r["output"] = normalize_text(r["output"])

random.seed(SEED)
random.shuffle(raw)

n = len(raw)
val_n = max(1, int(n * EVAL_SPLIT))
val_data = raw[:val_n]
train_data = raw[val_n:]

dataset = DatasetDict({
    "train": datasets.Dataset.from_list(train_data),
    "validation": datasets.Dataset.from_list(val_data)
})

dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 752
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 83
    })
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

def build_source(texto: str) -> str:
    return INSTR_PROMPT.format(texto=texto)

def preprocess_batch(batch):
    # batch["input"] é uma lista de strings
    sources = [build_source(t) for t in batch["input"]]
    targets = batch["output"]

    model_inputs = tokenizer(
        sources,
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding=False,
    )
    # use o argumento oficial para targets
    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LEN,
        truncation=True,
        padding=False,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = dataset.map(
    preprocess_batch,
    batched=True,
    remove_columns=dataset["train"].column_names
)

tokenized

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/752 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 752
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 83
    })
})

In [5]:
bertscore = evaluate.load("bertscore")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [[l.strip()] for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # Substituir -100 por pad_token_id para decodificação
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # BERTScore entre saída do modelo (predictions) e texto alvo (references)
    bert_result = bertscore.compute(
        predictions=decoded_preds,
        references=[l[0] for l in decoded_labels],
        lang="pt",          # importante para português
        rescale_with_baseline=True
    )

    precision = float(np.mean(bert_result["precision"]))
    recall    = float(np.mean(bert_result["recall"]))
    f1        = float(np.mean(bert_result["f1"]))

    return {
        "bertscore_precision": round(precision, 4),
        "bertscore_recall": round(recall, 4),
        "bertscore_f1": round(f1, 4),   # normalmente essa é a principal
    }

In [6]:
def load_lora_model():
    # Quantização 4-bit/8-bit (QLoRA)
    kwargs = dict(
        device_map="auto",
        load_in_4bit=USE_4BIT,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if BF16 else torch.float16,
    ) if torch.cuda.is_available() else {}

    base = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, **kwargs)
    base = prepare_model_for_kbit_training(base)

    lora_cfg = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=["q", "v", "k", "o", "wi", "wo"],  # nomes comuns em T5
        bias="none",
        task_type="SEQ_2_SEQ_LM",
    )
    peft_model = get_peft_model(base, lora_cfg)
    peft_model.print_trainable_parameters()
    return peft_model

model = load_lora_model()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1614: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens', 'encoder.embed_tokens']
  warnings.warn(


trainable params: 6,488,064 || all params: 229,391,616 || trainable%: 2.8284


In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    save_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_bertscore_f1",
    greater_is_better=True,

    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,

    fp16=FP16 and not BF16,
    bf16=BF16,

    predict_with_generate=True,
    generation_max_length=MAX_TARGET_LEN,
    generation_num_beams=4,

    seed=SEED,
    report_to=["none"],  # mude para ["tensorboard"] se quiser
)
set_seed(SEED)

In [8]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

train_result = trainer.train()

/tmp/ipykernel_577/310220481.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Bertscore Precision,Bertscore Recall,Bertscore F1
50,5.709300,1.226335,0.621300,0.636800,0.628400
100,1.167800,0.775525,0.752500,0.731900,0.741800
150,0.858400,0.637874,0.791800,0.751500,0.770700
200,0.724200,0.576302,0.814700,0.777700,0.795300
250,0.615400,0.533193,0.829400,0.802700,0.815300
300,0.553200,0.501034,0.836300,0.821100,0.828400
350,0.491900,0.500221,0.851600,0.823800,0.836900
400,0.454500,0.490571,0.864100,0.835500,0.849000
450,0.420600,0.478745,0.847900,0.836300,0.841900
500,0.410600,0.475219,0.849100,0.832600,0.840600


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has

In [9]:
metrics = train_result.metrics
metrics["train_samples"] = len(tokenized["train"])

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

# Avaliar no validation
eval_metrics = trainer.evaluate()
eval_metrics["eval_samples"] = len(tokenized["validation"])
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)

# Salvar (se LoRA, salva adaptadores; se FT, salva modelo completo)
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

***** train metrics *****
  epoch                    =       30.0
  total_flos               =  2419521GF
  train_loss               =     0.9067
  train_runtime            = 0:16:47.26
  train_samples            =        752
  train_samples_per_second =     22.397
  train_steps_per_second   =      0.715


***** eval metrics *****
  epoch                    =       30.0
  eval_bertscore_f1        =      0.849
  eval_bertscore_precision =     0.8641
  eval_bertscore_recall    =     0.8355
  eval_loss                =     0.4906
  eval_runtime             = 0:00:15.81
  eval_samples             =         83
  eval_samples_per_second  =      5.248
  eval_steps_per_second    =      0.379


('./outputs_ptt5v2/tokenizer_config.json',
 './outputs_ptt5v2/special_tokens_map.json',
 './outputs_ptt5v2/spiece.model',
 './outputs_ptt5v2/added_tokens.json',
 './outputs_ptt5v2/tokenizer.json')

In [10]:
def predict(texto: str, max_new_tokens=64, num_beams=4):
    inp = INSTR_PROMPT.format(texto=normalize_text(texto.lower()))
    tokens = tokenizer(inp, return_tensors="pt", truncation=True, max_length=MAX_INPUT_LEN).to(model.device)
    
    with torch.no_grad():
        out = model.generate(
            **tokens,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            length_penalty=0.9,
            early_stopping=True,
        )
        
    return tokenizer.decode(out[0], skip_special_tokens=True).strip()

exemplo = 'INSTITUI O PROGRAMA “VOLTAR A ESTUDAR MUDA TUDO”, COM O OBJETIVO DE PROMOVER CAMPANHAS DE INCENTIVO À MATRÍCULA E VALORIZAÇÃO DA EDUCAÇÃO DE JOVENS E ADULTOS (EJA), NO ÂMBITO DO MUNICÍPIO DE NATAL/RN.'
# GPT Output: Promover campanhas de incentivo à matrícula e valorização da Educação de Jovens e Adultos (EJA) no município de Natal.
print(predict(exemplo))

Criar o programa “voltar a estudar muda tudo” com campanhas de incentivo à matrícula e valorização da educação de jovens e adultos no município.


In [11]:
from peft import PeftModel

fused_dir = OUTPUT_DIR + "-merged"

base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16 if FP16 else torch.float32)
peft_loaded = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

merged = peft_loaded.merge_and_unload()
merged.save_pretrained(fused_dir)
tokenizer.save_pretrained(fused_dir)

print(f"Modelo fundido salvo em: {fused_dir}")

Modelo fundido salvo em: ./outputs_ptt5v2-merged


In [12]:
from huggingface_hub import login

login()

In [13]:
from huggingface_hub import HfApi

api = HfApi()

MODEL_REPO = "thiagoambiel/ptt5v2-pl-text2action"
api.create_repo(MODEL_REPO, repo_type="model", private=True, exist_ok=True)

MODEL_REPO

'thiagoambiel/ptt5v2-pl-text2action'

In [14]:
from huggingface_hub import upload_folder

upload_folder(
    folder_path="/kaggle/working/outputs_ptt5v2-merged",
    repo_id=MODEL_REPO,
    repo_type="model",
    commit_message="PTT5-v2 LoRA com BERTScore 0.84"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/thiagoambiel/ptt5v2-pl-text2action/commit/3e4a754d75fbe031d9b1fcc1ce894ffe61f5d66c', commit_message='PTT5-v2 LoRA com BERTScore 0.84', commit_description='', oid='3e4a754d75fbe031d9b1fcc1ce894ffe61f5d66c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thiagoambiel/ptt5v2-pl-text2action', endpoint='https://huggingface.co', repo_type='model', repo_id='thiagoambiel/ptt5v2-pl-text2action'), pr_revision=None, pr_num=None)